# 3. 없다면 어떻게 될까 (ResNet Ablation Study) [프로젝트]

## 3-1. 프로젝트: ResNet Ablation Study

### 0) 라이브러리 버전 확인하기
---
사용할 라이브러리 버전을 둘러봅시다.

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.6.0
1.21.4


### 1) ResNet 기본 블록 구성하기
---
이제 실전으로 돌아와서 ResNet-34와 ResNet-50 네트워크를 직접 만든다고 생각해 봅시다.



In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Conv2D, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D, Dense 등 필요한 모든 Keras 레이어들을 import
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D, Dense

In [3]:
# resnet block
def build_resnet_block(input_layer,
                       num_cnn,
                       channel,
                       block_num,
                       is_50
                      ):
    x = input_layer
    
    for i in range(num_cnn):
        residual = x # 잔차 연결을 위한 저장
    
        layer_name = f'stage{block_num+2}_{i+1}'
        
        if is_50: # resnet-50
            # 1*1 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                kernel_size=(1,1),
                                padding='same',
                                name=f'{layer_name}_1x1')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        
            # 3*3 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                    kernel_size=(3,3),
                                    padding='same',
                                    name=f'{layer_name}_3x3')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)

            # 1*1 conv 레이어
            x = keras.layers.Conv2D(filters=channel*4,
                                    kernel_size=(1,1),
                                    padding='same',
                                    name=f'{layer_name}_1x1_2')(x)
            x = layers.BatchNormalization()(x)
        
        else: # resnet-34
            # 3*3 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                    kernel_size=(3,3),
                                    padding='same',
                                    name=f'{layer_name}_1')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)

            # 3*3 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                    kernel_size=(3,3),
                                    padding='same',
                                    name=f'{layer_name}_2')(x)
            x = layers.BatchNormalization()(x)
    
        # 잔차 연결: 채널 수가 다르면 1x1 Conv를 이용해 크기 맞추기
        if residual.shape[-1] != x.shape[-1]:
            residual = layers.Conv2D(filters=channel * 4 if is_50 else channel, 
                                      kernel_size=(1, 1), 
                                      padding='same',
                                     name=f'{layer_name}_residual_conv')(residual)
            residual = layers.BatchNormalization()(residual)

        # 잔차 연결
        x = layers.add([x, residual])
        x = layers.ReLU()(x)

    return x

In [4]:
# bulid resnet
def build_resnet(input_shape,
                 num_cnn_list = [3, 4, 6, 3],
                 channel_list = [64, 128, 256, 512],
                 is_50=False):
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    # conv1 (첫 번째 Conv Layer)
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # conv2 다운샘플링
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
     
    # config list들의 길이만큼 반복해서 블록을 생성
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = build_resnet_block(x,
                                num_cnn=num_cnn,
                                channel=channel,
                                block_num=i,
                                is_50=is_50)
    
    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully connected (Dense) Layer
    x = layers.Dense(1000, activation='softmax')(x)
    
    # Define the Model
    model = models.Model(inputs=input_layer, outputs=x)
    return model

### 2) ResNet-34, ResNet-50 Complete Model
---

ResNet 모델 구현 시 Sequential API나 Subclass API를 사용한다면, 그 과정에서 모델 단위로 기존의 코드를 재활용했을 때 model.summary() 호출 시 서브모델 내부의 레이어 구성이 생략되고 서브모델 단위로만 출력될 우려가 있습니다. 모델 구성만을 위해서는 그런 방법도 무방하지만, 가급적 이번 실습에서는 VGG 실습 예시에서처럼 Functional API 를 구성하는 방식을 사용할 것을 권합니다.

**ResNet-34**

VGG와 같이 블록을 만드는 함수를 사용해서 직접 전체 모델을 만들어 봅시다. ResNet-34와 ResNet-50의 차이에 따라 달라지는 구성(configuration)을 함수에 전달해서 같은 생성 함수 build_resnet()를 통해서 ResNet의 여러 가지 버전들을 모두 만들어 낼 수 있도록 해야 합니다.

In [5]:
resnet_34 = build_resnet(input_shape=(32, 32, 3), is_50 = False)
resnet_34.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 16, 16, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

**ResNet-50**

In [6]:
resnet_50 = build_resnet(input_shape=(32, 32, 3), is_50=True)
resnet_50.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 16, 16, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_33 (ReLU)                 (None, 16, 16, 64)   0           batch_normalization_36[0][0]     
____________________________________________________________________________________________

### 3) 일반 네트워크(plain network) 만들기
---

**블록 코드 수정하기**

우리는 앞에서 ResNet 모델을 구현했습니다. ResNet의 핵심 아이디어는 skip connection과 residual network기 때문에, ResNet의 효과를 보여주기 위해서는 skip connection이 없는 일반 네트워크(plain net)가 필요합니다. 위에서 ResNet 블록을 만들기 위한 함수를 그대로 활용해서 skip connection이 없는 블록을 만들 수 있도록 기능을 추가해 주세요!

In [7]:
# plain block
def build_plainnet_block(input_layer,
                         num_cnn,
                         channel,
                         block_num,
                         is_50,
                         plain
                      ):
    x = input_layer
    
    for i in range(num_cnn):
        residual = x # 잔차 연결을 위한 저장
        
        layer_name = f'stage{block_num+2}_{i+1}'
        if is_50: # net-50
            # 1*1 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                kernel_size=(1,1),
                                padding='same',
                                name=f'{layer_name}_1')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        
            # 3*3 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                    kernel_size=(3,3),
                                    padding='same',
                                    name=f'{layer_name}_2')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)

            # 1*1 conv 레이어
            x = keras.layers.Conv2D(filters=channel*4,
                                    kernel_size=(1,1),
                                    padding='same',
                                    name=f'{layer_name}_3')(x)
            x = layers.BatchNormalization()(x)
            if plain:
                x = layers.ReLU()(x)
        
        else: # net-34
            # 3*3 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                    kernel_size=(3,3),
                                    padding='same',
                                    name=f'{layer_name}_1')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
            
            # 3*3 conv 레이어
            x = keras.layers.Conv2D(filters=channel,
                                    kernel_size=(3,3),
                                    padding='same',
                                    name=f'{layer_name}_2')(x)
            x = layers.BatchNormalization()(x)
            if plain:
                x = layers.ReLU()(x)
                
        if plain == False:
            # 잔차 연결: 채널 수가 다르면 1x1 Conv를 이용해 크기 맞추기
            if residual.shape[-1] != x.shape[-1]:
                residual = layers.Conv2D(filters=channel * 4 if is_50 else channel, 
                                          kernel_size=(1, 1), 
                                          padding='same',
                                         name=f'{layer_name}_residual_conv')(residual)
                residual = layers.BatchNormalization()(residual)

            # 잔차 연결
            x = layers.add([x, residual])
            x = layers.ReLU()(x)
    return x

**전체 함수 코드 수정하기**

이제 위에서 만든 블록 함수를 토대로 전체 네트워크를 만들 수 있도록 전체 네트워크 코드를 수정합시다. ResNet-50과 ResNet-34, 그리고 같은 레이어를 가지지만 skip connection이 없는 PlainNet-50과 PlainNet-34를 만들 수 있는 함수 build_plainnet()를 만들어 보세요. 이때 입력 이미지의 크기는 (224, 224, 3)으로 해주세요.

In [8]:
# bulid plainnet
def build_plainnet(input_shape=(224,224,3),
                   num_cnn_list = [3, 4, 6, 3],
                   channel_list = [64, 128, 256, 512],
                   is_50=False,
                   plain=False):
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    # conv1 (첫 번째 Conv Layer)
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # conv2 다운샘플링
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
     
    # config list들의 길이만큼 반복해서 블록을 생성
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = build_plainnet_block(x,
                               num_cnn=num_cnn,
                               channel=channel,
                               block_num=i,
                               is_50=is_50,
                               plain=plain)
    
    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully connected (Dense) Layer
    x = layers.Dense(1000, activation='softmax')(x)
    
    # Define the Model
    model = models.Model(inputs=input_layer, outputs=x)
    return model

### 4) ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34
---
**Ablation Study**

이제 VGG-16, 19 예제와 같이 ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34에 대해서 학습을 진행해 봅니다. 그리고 결과를 비교해 봅시다! ResNet은 많은 레이어와 Pooling을 거치므로 CIFAR-10에서는 오버피팅(overfitting)으로 잘 동작하지 않을 수 있습니다. 레이어가 많고 학습해야 할 변수(parameter)가 많은 데 비해, 데이터 수가 많지 않기 때문이지요. 224x224 픽셀 크기의 데이터셋을 찾아서 실험해 보도록 합시다. 학습은 끝까지 시키기엔 시간이 없으니 확인을 위한 정도의 epoch로 설정해 주세요.

어떤 데이터셋을 사용하셔도 무방하지만, 얼른 떠오르는 것이 없다면 tensorflow-datasets에서 제공하는 **cats_vs_dogs 데이터셋**을 추천합니다. 아마 이 데이터셋도 다루어 보신 적이 있을 것입니다. Tensorflow에서 제공하는 데이터셋이므로 오늘 VGG 학습에 사용했던 CIFAR-10을 로딩하는 것과 같은 방법으로 활용하실 수 있습니다.

In [9]:
! pip install tensorflow-datasets

In [10]:
import tensorflow_datasets as tfds

In [11]:
BATCH_SIZE = 4
EPOCH = 5

In [12]:
# 새로운 URL 설정
new_url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

# cats_vs_dogs 데이터셋의 URL을 덮어쓰기
setattr(tfds.image_classification.cats_vs_dogs, '_URL', new_url)

In [13]:
# cats_vs_dogs 데이터셋을 불러오기
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],  # 80%는 훈련용, 20%는 검증용
    as_supervised=True,  # (이미지, 라벨) 형식으로 반환
    shuffle_files=True,  # 파일을 섞어서 읽기
    with_info=True,  # 데이터셋 정보도 반환
)

# ds_train, ds_test로 저장
print(f"훈련 데이터셋: {ds_train}")
print(f"테스트 데이터셋: {ds_test}")

훈련 데이터셋: <_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>
테스트 데이터셋: <_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>


In [14]:
# 데이터셋 정보 확인
print(ds_info.features)

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})


In [15]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

tf.Tensor(18610, shape=(), dtype=int64)
tf.Tensor(4652, shape=(), dtype=int64)


In [16]:
def normalize_and_resize_img(image, label, target_size=(224, 224)):
    """이미지를 리사이즈하고 정규화하는 함수."""
    # 이미지 차원 확인: (height, width, channels)
    image = tf.image.resize(image, target_size)  # 이미지 크기를 (224, 224)로 리사이즈
    image = tf.cast(image, tf.float32) / 255.0   # 0~255 범위를 0~1로 정규화
    return image, label

In [17]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [18]:
# 이미지 데이터 정규화
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [19]:
resnet_34 = build_plainnet(is_50=False, plain=False)
resnet_34.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_89 (BatchNo (None, 112, 112, 64) 256         conv2d_2[0][0]                   
__________________________________________________________________________________________________
re_lu_82 (ReLU)                 (None, 112, 112, 64) 0           batch_normalization_89[0][0]     
____________________________________________________________________________________________

In [20]:
resnet_50 = build_plainnet(is_50=True, plain=False)
resnet_50.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 112, 112, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_125 (BatchN (None, 112, 112, 64) 256         conv2d_3[0][0]                   
__________________________________________________________________________________________________
re_lu_115 (ReLU)                (None, 112, 112, 64) 0           batch_normalization_125[0][0]    
____________________________________________________________________________________________

In [21]:
plainnet_34 = build_plainnet(is_50=False, plain=True)
plainnet_34.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_178 (Bat (None, 112, 112, 64)      256       
_________________________________________________________________
re_lu_164 (ReLU)             (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
stage2_1_1 (Conv2D)          (None, 56, 56, 64)        36928     
_________________________________________________________________
batch_normalization_179 (Bat (None, 56, 56, 64)        256 

In [22]:
plainnet_50 = build_plainnet(is_50=True, plain=True)
plainnet_50.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_211 (Bat (None, 112, 112, 64)      256       
_________________________________________________________________
re_lu_197 (ReLU)             (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
stage2_1_1 (Conv2D)          (None, 56, 56, 64)        4160      
_________________________________________________________________
batch_normalization_212 (Bat (None, 56, 56, 64)        256 

In [ ]:
resnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

hist_res_34 = resnet_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples * 0.8/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train'].num_examples * 0.2/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

/opt/conda/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/15
1435/4652 [========>.....................] - ETA: 16:24 - loss: 0.7973 - accuracy: 0.5934

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


1688/4652 [=========>....................] - ETA: 15:07 - loss: 0.7754 - accuracy: 0.5983

1698/4652 [=========>....................] - ETA: 15:04 - loss: 0.7740 - accuracy: 0.5992

...

4408/4652 [===========================>..] - ETA: 1:14 - loss: 0.6772 - accuracy: 0.6422

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


4652/4652 [==============================] - ETA: 0s - loss: 0.6721 - accuracy: 0.6458

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9


4652/4652 [==============================] - 1611s 334ms/step - loss: 0.6721 - accuracy: 0.6458 - val_loss: 0.6742 - val_accuracy: 0.6692
Epoch 2/15
1138/4652 [======>.......................] - ETA: 17:58 - loss: 0.5587 - accuracy: 0.7142

In [ ]:
resnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

hist_res_50 = resnet_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples * 0.8/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train'].num_examples * 0.2/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plainnet_32.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

hist_pln_32 = plainnet_32.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples * 0.8/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train'].num_examples * 0.2/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plainnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

hist_pln_50 = plainnet_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples * 0.8/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train'].num_examples * 0.2/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

**시각화**

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(121)
plt.plot(hist_res_34.history['loss'], 'r')
plt.plot(hist_res_50.history['loss'], 'b')
plt.plot(hist_pln_34.history['loss'], 'o')
plt.plot(hist_pln_50.history['loss'], 'k')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'resnet_50', 'plainnet_34', 'plainnet_50'], loc='upper left')

plt.subplot(122)
plt.plot(hist_res_34.history['val_accuracy'], 'r')
plt.plot(hist_res_50.history['val_accuracy'], 'b')
plt.plot(hist_pln_34.history['val_accuracy'], 'o')
plt.plot(hist_pln_50.history['val_accuracy'], 'k')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'resnet_50', 'plainnet_34', 'plainnet_50'], loc='upper left')
plt.show()